In [4]:
from networkit import *
import os
import numpy as np
import sys

## Read graph and coordinates

In [5]:
graphPath = "/home/harry/geographer-dev/meshes/NLR.graph"
G = readGraph(graphPath, Format.METIS)
print( G.numberOfNodes(), G.numberOfEdges() )

4163763 12487976


In [8]:
coordsFile = graphPath+".xyz"
coordsAll = [line.rstrip('\n') for line in open(coordsFile) if not line.startswith('%')]
print( coordsAll[:10] )

['11.65750000 0.00000000', '11.65750000 0.00001838', '11.65750000 0.00004104', '11.65750000 0.00007238', '11.65750000 0.00011698', '11.65750000 0.00017941', '11.65750000 0.00026403', '11.65750000 0.00037547', '11.65750000 0.00051992', '11.65750000 0.00070690']


In [9]:
coordsX = [ float(x.split(' ')[0]) for x in coordsAll ]
print( max(coordsX) )

11.6575


In [10]:
coordsY = [ float(x.split(' ')[1]) for x in coordsAll ]
print( max(coordsY) )

10.5977


## Read partitions

In [60]:
#read partitions

partFile = "/home/harry/geographer-dev/partResults/partKM8.out"
partitionKM = [line.rstrip('\n') for line in open(partFile)]

In [61]:
partFile = "/home/harry/geographer-dev/partResults/partHKM8.out"
partitionHKM =  [line.rstrip('\n') for line in open(partFile)]

In [62]:
print( len(partitionKM) )
#print( len(partitionHKM_h1) )

15037


In [63]:
print( partitionKM[:10] )
print( partitionHKM[:10] )

['% 15036', '2', '2', '4', '4', '4', '2', '7', '7', '7']
['% 15036', '2', '2', '4', '4', '4', '2', '6', '6', '7']


In [64]:
# turn to int, remove first line

#partitionHKM_h1 = [ int(x) for x in partitionHKM_h1[1:] ]
#partitionHKM_h2 = [ int(x) for x in partitionHKM_h2[1:] ]
#assert( len(partitionHKM_h1)==len(partitionHKM_h2) )

partitionHKM = [ int(x) for x in partitionHKM[1:] ]
partitionKM = [ int(x) for x in partitionKM[1:] ]



In [54]:
# read the partitions for the different hierarchy levels

levels = 3
allHier = list()
for i in range(1,levels):
    partFile = "/home/harry/geographer-dev/partResults/partHKM10_h" + str(i) + ".out"
    allHier.append( [line.rstrip('\n') for line in open(partFile)] )
    allHier[-1] = [ int(x) for x in allHier[-1] ] 
print( allHier[0][0:10] )    
print( allHier[1][0:10] )    

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## Export data to gephi

In [13]:
client = gephi.streaming.GephiStreamingClient(url="http://localhost:8080/workspace1")

In [14]:
client.exportGraph(G)

Could not connect to the gephi streaming plugin.
Did you start the streaming master server in gephi and provide the name of your workspace?
If the workspace is named 'Workspace 0', the corresponding url is http://localhost:8080/workspace0 (adapt port)


In [12]:
client.exportNodeValues( G, coordsX, "X")
client.exportNodeValues( G, coordsY, "Y")

Could not connect to the gephi streaming plugin.
Did you start the streaming master server in gephi and provide the name of your workspace?
If the workspace is named 'Workspace 0', the corresponding url is http://localhost:8080/workspace0 (adapt port)
Could not connect to the gephi streaming plugin.
Did you start the streaming master server in gephi and provide the name of your workspace?
If the workspace is named 'Workspace 0', the corresponding url is http://localhost:8080/workspace0 (adapt port)


In [69]:
client.exportNodeValues(G, partitionKM, "KM")
client.exportNodeValues(G, partitionHKM, "HKM")

#client.exportNodeValues(G, allHier[0], "HKM_h1")
#client.exportNodeValues(G, allHier[1], "HKM_h2")

In [40]:
print( len(allHier) )
for i in range(levels-1):
    print(i)
    client.exportNodeValues(G, allHier[i], "HKM_h"+str(i))

2
0
1
